## 인천 나리타 노선 가격 정보 읽어오기
- 대상 사이트 : 아시아나항공 국제선 예매 사이트

### 대한항공 사이트 웹 크롤링
    - 아시아나항공 국제선 항공권예매 페이지를 이용한 크롤링
    - 사이트 접속 데이터 크롤하여 BeautifulSoup 파싱
    - 필요 데이터 추출 하여 Pandas 의 DataFrame 형태로 생성
    - 생성된 데이터 엑셀 파일로 저장

In [155]:
from IPython.display import display
import requests
import pandas as pd
import numpy as np
from pandas import DataFrame
from bs4 import BeautifulSoup
import time
from datetime import datetime
from datetime import timedelta
import json
from common.crawling_util import simple_crawling
from common.parsing_util import stat_fare, mining_value_by_last_field_name
from common.util import find_between

def crawling_OZ_data(dpt,arr,dpt_date):
    ##출발지, 도착지, 출발일을 기준으로 국내선(국제선) 편도 가격 읽어오기
    print('Crawling asiana homepage schedule site')
    url = "https://flyasiana.com/I/ko/RevenueInternationalFareDrivenFlightSelect.do"
    head = {
        'Referer':'https://flyasiana.com/I/ko/RevenueInternationalFareDrivenCalendarSelect.do',
        'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    param ={
        'hidPageType':'',           'domIntType':'I',
        'tripType':'OW',            'departureAirportC':'',
        'arrivalAirportC':'',       'departureDateC':'',
        'arrivalDateC':'',          'openDepartureAirportC':'',
        'openArrivalAirportC':'',   'multiSegInfo':'',
        #'sessionUniqueKey:a8c45052-bbf9-4d67-9c66-bd4614ed034b
        'officeId':'',              'entType':'',
        'corporateCode':'',         'couponDesc':'',
        'cServiceType':'',          'couponCode':'',
        'query':'',
        'departureArea':'KR',       'departureAirport':dpt,
        'arrivalArea':'JP',         'arrivalAirport':arr,
        'departureDate':dpt_date,#'2017-06-15',
        'adultCount':'1',     'childCount':'0',     'infantCount':'0',
        'ageCalYear':'2017',  'ageCalMonth':'1',    'ageCalDay':'0',
        'cabinClass':'T',     'fareViewType':'L',
    }

    return simple_crawling(url,param,head=head,method='post',json=False)

def get_json_data(raw_data):
    ## 유류세, 공항세 정보
    start_txt = "var fareFamilyWithAllAvail = eval('"
    end_txt = "');"
    find_data = find_between(raw_data,start_txt,end_txt)
    return {'json_data': eval(find_data.replace('true',"True").replace('false',"False"))}

## 국내선 텍스 정보 리스트 읽어오기 => 국제선 용으로 변경 필요???
def get_tax_info(raw_data):
    ## 유류세, 공항세 정보
    start_txt = "var fareFamilyWithAllAvail = eval('"
    end_txt = "');"
    find_data = find_between(raw_data,start_txt,end_txt)
    fare_data = eval(find_data.replace('true',"True").replace('false',"False"))
    ## 유류세, 텍스 정보
    return [fare_data[0]['fareFamilyAmounts'][0]['fuelCharge'],fare_data[0]['fareFamilyAmounts'][0]['totalTax']]

def raw_json_to_df(raw_data):
    data_heads = ['date','flt','dpt','arr','fare','tax1','tax2','seat']
    json_data = get_json_data(raw_data)
    sch_list = []
    for l_data in json_data['json_data']:
        ## fare 정보 찾아
        fare_data = l_data['fareFamilyAmounts'][0]['itineraryAvailDataOfRecommend']['paxTypeFareDatas'][0]
        ## flight 정보 찾아오기
        for flt_info in l_data['fareFamilyAmounts'][0]['itineraryAvailDataOfRecommend']['availDataList'][0]:
            l_list = []
            flt_data = flt_info['flightInfoDatas'][0]
            ## flight 정보 추가
            l_list.extend([flt_data['departureDate'][:8],flt_data['flightNo'],flt_data['departureDate'][8:12],flt_data['arrivalDate'][8:12]])
            ## fare 정보 추가
            l_list.extend([fare_data['amountWithoutTax'],fare_data['fuelCharge'],fare_data['totalTax']])
            ## seat 정보 추가
            l_list.append(flt_info['seatCount'] if flt_info['seatCount'] != '' else '0')
            sch_list.append(l_list)
    return pd.DataFrame(sorted(sch_list),columns=data_heads)

## raw 데이터로 부터 DataFrame 생성
def raw_html_to_df(dpt_date,raw_data):
    data_heads = ['date','flt','dpt','arr','fare1','fare2','fare3','tax1','tax2','seat']
    soup = BeautifulSoup(raw_data,'lxml')
    sch_list = []
    tax_info = get_tax_info(raw_data);
    ## fare information 읽어옥
    fare_list = []
    for tr in soup.select('#tbFareFamily tbody tr'):
        td_list = []
        td = tr.select('td')
        #td_list.append(td[0].select('input')[0]['value'])
        td_list.append(json.loads(td[0].select('input')[0]['value'].replace("'",'"')))
        td_list.append(td[1].text[:-2])
        fare_list.append(td_list)
    ## flight information 읽어오기
    ## 읽어온 fare 정보를 이용하여 해당 flight 정보의 가격정보 매핑 처리
    for tr in soup.select('#tdFlightList0 tbody tr'):
        td = tr.select('td')[:3]
        td_list = [dpt_date]
        flt_no = td[0].text.strip()
        td_list.append(flt_no)
        td_list.append(td[1].text[-5:].replace(':',''))
        td_list.append(td[2].text[-5:].replace(':',''))
        td_list.extend([fare[1].replace(',','')
                            if flt_no[2:] in fare[0] and fare[0][flt_no[2:]] != '0' else '0'
                            for fare in fare_list])
        ## tax 정보 읽어오는 부분 처리 추가
        td_list.extend(tax_info)
        td_list.append(fare_list[-1][0][flt_no[2:]])
        sch_list.append(td_list)
    #return sch_list
    return pd.DataFrame(sch_list,columns=data_heads)

## 하루 기준 가격정보, 텍스정보, 최소값, 최대값, 평균 DataFrame 생성
def read_OZ_1day_fare(dpt,arr,dpt_date):
    ## 데이터 읽어오기
    raw_data = crawling_OZ_data(dpt,arr,dpt_date)
    df=raw_json_to_df(raw_data)
    if df is None or len(df) == 0: ## 읽는 도중 에러가 나거나 익셉션 발생으로 문제가 있을 경우 처리
        print('********** No Data **********')
        return None
    ## 최소값 최대값 평균 계산
    stat = stat_fare(df,columns=['fare'])
    ## 요약정보 추가
    df.ix[len(df)] = [dpt_date,'min','max','mean',str(int(stat[0])),str(int(stat[1])),str(int(stat[2])),'']
    return df

## 정해진 기간의 데이터 읽어오기, 기본 30일
def read_OZ_date_range_fare(dpt,arr,start=0,end=31):
    # 기본 30일간 데이터 읽어서 파일로 저장
    date_range = [ (datetime.today()+timedelta(1)*i).strftime('%Y%m%d') for i in range(start,end)]
    fare_list = []
    for d in date_range:
        try:
            fare_df = read_OZ_1day_fare(dpt,arr,d)
            if fare_df is not None:
                fare_list.append(fare_df)
        except Exception as e:
            print('****** Error occured : ',e)
    result = pd.concat(fare_list,ignore_index=True)
    print('++++++++++Total : ', len(result))
    ## 파일 저장
    result.to_excel('{}/{}_{}_{}_{}_{}_{}.xls'.format('excel','OZ',dpt,arr,start,end,datetime.today().strftime('%Y%m%d%H%M')))
    return result

In [156]:
## 하루치 데이터 읽어오기
dpt, arr, dpt_date = 'ICN','NRT','20170526'
read_OZ_1day_fare(dpt,arr,dpt_date)

Crawling asiana homepage schedule site
Start Simple crawling :  https://flyasiana.com/I/ko/RevenueInternationalFareDrivenFlightSelect.do
>> Parameters
hidPageType: , arrivalAirportC: , entType: , ageCalYear:2017 , openArrivalAirportC: , query: , cServiceType: , officeId: , couponCode: , tripType:OW , departureArea:KR , departureDateC: , cabinClass:T , arrivalAirport:NRT , ageCalDay:0 , multiSegInfo: , arrivalDateC: , departureAirportC: , openDepartureAirportC: , arrivalArea:JP , fareViewType:L , infantCount:0 , departureDate:20170526 , departureAirport:ICN , adultCount:1 , couponDesc: , ageCalMonth:1 , corporateCode: , childCount:0 , domIntType:I , 
End Simple crawling


,date,flt,dpt,arr,fare,tax1,tax2,seat
0,20170526,102,0900,1110,187200.00,0.00,28000.00,9
1,20170526,102,0900,1110,199200.00,0.00,28000.00,9
2,20170526,102,0900,1110,289200.00,0.00,28000.00,9
3,20170526,1023,1545,1755,199200.00,0.00,28000.00,1
4,20170526,1023,1545,1755,289200.00,0.00,28000.00,3
5,20170526,104,1000,1220,187200.00,0.00,28000.00,9
6,20170526,104,1000,1220,199200.00,0.00,28000.00,9
7,20170526,104,1000,1220,289200.00,0.00,28000.00,9
8,20170526,106,1525,1750,187200.00,0.00,28000.00,9
9,20170526,106,1525,1750,199200.00,0.00,28000.00,9


In [157]:
## 정해진 기간의 데이터 읽어오기
dpt, arr = 'ICN','NRT'
start,end =1, 31 ## 읽어올 데이터 기간
read_OZ_date_range_fare(dpt,arr,start,end)

Crawling asiana homepage schedule site
Start Simple crawling :  https://flyasiana.com/I/ko/RevenueInternationalFareDrivenFlightSelect.do
>> Parameters
hidPageType: , arrivalAirportC: , entType: , ageCalYear:2017 , openArrivalAirportC: , query: , cServiceType: , officeId: , couponCode: , tripType:OW , departureArea:KR , departureDateC: , cabinClass:T , arrivalAirport:NRT , ageCalDay:0 , multiSegInfo: , arrivalDateC: , departureAirportC: , openDepartureAirportC: , arrivalArea:JP , fareViewType:L , infantCount:0 , departureDate:20170524 , departureAirport:ICN , adultCount:1 , couponDesc: , ageCalMonth:1 , corporateCode: , childCount:0 , domIntType:I , 
End Simple crawling
Crawling asiana homepage schedule site
Start Simple crawling :  https://flyasiana.com/I/ko/RevenueInternationalFareDrivenFlightSelect.do
>> Parameters
hidPageType: , arrivalAirportC: , entType: , ageCalYear:2017 , openArrivalAirportC: , query: , cServiceType: , officeId: , couponCode: , tripType:OW , departureArea:KR , d

,date,flt,dpt,arr,fare,tax1,tax2,seat
0,20170524,102,0900,1110,187200.00,0.00,28000.00,9
1,20170524,102,0900,1110,199200.00,0.00,28000.00,9
2,20170524,102,0900,1110,289200.00,0.00,28000.00,9
3,20170524,1023,1620,1830,187200.00,0.00,28000.00,9
4,20170524,1023,1620,1830,199200.00,0.00,28000.00,9
5,20170524,1023,1620,1830,289200.00,0.00,28000.00,9
6,20170524,104,1000,1220,187200.00,0.00,28000.00,9
7,20170524,104,1000,1220,199200.00,0.00,28000.00,9
8,20170524,104,1000,1220,289200.00,0.00,28000.00,9
9,20170524,106,1525,1750,187200.00,0.00,28000.00,9


In [158]:
## 정해진 기간의 데이터 읽어오기
dpt, arr = 'ICN','NRT'
start,end =31, 45 ## 읽어올 데이터 기간
read_OZ_date_range_fare(dpt,arr,start,end)

Crawling asiana homepage schedule site
Start Simple crawling :  https://flyasiana.com/I/ko/RevenueInternationalFareDrivenFlightSelect.do
>> Parameters
hidPageType: , arrivalAirportC: , entType: , ageCalYear:2017 , openArrivalAirportC: , query: , cServiceType: , officeId: , couponCode: , tripType:OW , departureArea:KR , departureDateC: , cabinClass:T , arrivalAirport:NRT , ageCalDay:0 , multiSegInfo: , arrivalDateC: , departureAirportC: , openDepartureAirportC: , arrivalArea:JP , fareViewType:L , infantCount:0 , departureDate:20170623 , departureAirport:ICN , adultCount:1 , couponDesc: , ageCalMonth:1 , corporateCode: , childCount:0 , domIntType:I , 
End Simple crawling
Crawling asiana homepage schedule site
Start Simple crawling :  https://flyasiana.com/I/ko/RevenueInternationalFareDrivenFlightSelect.do
>> Parameters
hidPageType: , arrivalAirportC: , entType: , ageCalYear:2017 , openArrivalAirportC: , query: , cServiceType: , officeId: , couponCode: , tripType:OW , departureArea:KR , d

,date,flt,dpt,arr,fare,tax1,tax2,seat
0,20170623,102,0900,1110,187200.00,0.00,28000.00,9
1,20170623,102,0900,1110,199200.00,0.00,28000.00,9
2,20170623,102,0900,1110,289200.00,0.00,28000.00,9
3,20170623,104,1000,1220,187200.00,0.00,28000.00,9
4,20170623,104,1000,1220,199200.00,0.00,28000.00,9
5,20170623,104,1000,1220,289200.00,0.00,28000.00,9
6,20170623,106,1525,1750,187200.00,0.00,28000.00,9
7,20170623,106,1525,1750,199200.00,0.00,28000.00,9
8,20170623,106,1525,1750,289200.00,0.00,28000.00,9
9,20170623,108,1940,2200,187200.00,0.00,28000.00,9


In [159]:
dpt, arr = 'ICN','NRT'
start,end =45, 90 ## 읽어올 데이터 기간
read_OZ_date_range_fare(dpt,arr,start,end)

Crawling asiana homepage schedule site
Start Simple crawling :  https://flyasiana.com/I/ko/RevenueInternationalFareDrivenFlightSelect.do
>> Parameters
hidPageType: , arrivalAirportC: , entType: , ageCalYear:2017 , openArrivalAirportC: , query: , cServiceType: , officeId: , couponCode: , tripType:OW , departureArea:KR , departureDateC: , cabinClass:T , arrivalAirport:NRT , ageCalDay:0 , multiSegInfo: , arrivalDateC: , departureAirportC: , openDepartureAirportC: , arrivalArea:JP , fareViewType:L , infantCount:0 , departureDate:20170707 , departureAirport:ICN , adultCount:1 , couponDesc: , ageCalMonth:1 , corporateCode: , childCount:0 , domIntType:I , 
End Simple crawling
Crawling asiana homepage schedule site
Start Simple crawling :  https://flyasiana.com/I/ko/RevenueInternationalFareDrivenFlightSelect.do
>> Parameters
hidPageType: , arrivalAirportC: , entType: , ageCalYear:2017 , openArrivalAirportC: , query: , cServiceType: , officeId: , couponCode: , tripType:OW , departureArea:KR , d

,date,flt,dpt,arr,fare,tax1,tax2,seat
0,20170707,102,0900,1110,205200.00,0.00,28000.00,9
1,20170707,102,0900,1110,217200.00,0.00,28000.00,9
2,20170707,102,0900,1110,301200.00,0.00,28000.00,9
3,20170707,104,1000,1220,205200.00,0.00,28000.00,9
4,20170707,104,1000,1220,217200.00,0.00,28000.00,9
5,20170707,104,1000,1220,301200.00,0.00,28000.00,9
6,20170707,106,1525,1750,205200.00,0.00,28000.00,9
7,20170707,106,1525,1750,217200.00,0.00,28000.00,9
8,20170707,106,1525,1750,301200.00,0.00,28000.00,9
9,20170707,108,1850,2110,205200.00,0.00,28000.00,9
